In [2]:
import cirq
import numpy as np
import itertools

In [3]:
def PauliOperator(label):
    """Generates a specified Pauli operator
    
    Parameters:
        label (str): Pauli operator label (e.g. 'ZZZZIIIX')
        
    Returns:
        numpy matrix of corresponding Pauli operator
    """
    pauli = {
        'I': np.matrix([[1,0],[0,1]]),
        'Z': np.matrix([[1,0],[0,-1]]),
        'X': np.matrix([[0,1],[1,0]]),
        'Y': np.matrix([[0,-1j],[1j,0]])
    }
    
    operator = pauli[label[0]]
    for letter in label[1:]:
        operator = np.kron(operator, pauli[letter])

    return operator

#PauliOperator('XXX')

matrix([[0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0]])

In [4]:
def PauliDecompose(hmat):
    """Decompose a Hermitian matrix into a sum of Pauli matrices
    
    Parameters:
        hmat (matrix): hermitian matrix to decompose
        
    Returns:
        dictionary of {Pauli matrix(str) : coefficient (float)}
    """
    coeff = {}
    nbits = int(np.log2(hmat.shape[0]))
    labels = itertools.product('IXYZ', repeat=nbits)
    labels = [''.join(i) for i in labels] 
    for label in labels:
        tmp = np.matmul(hmat, PauliOperator(label))
        coeff[label] = np.trace(tmp) / hmat.shape[0]
    
    return coeff

# demo
PauliDecompose(PauliOperator('X') + PauliOperator('Z'))

{'I': 0.0, 'X': 1.0, 'Y': 0j, 'Z': 1.0}

In [5]:
def UGate(umat):
    """Realises the specified unitary digonal matrix in a Cirq quantum cricuit
    
    Parameters:
        umat (matrix): unitary diagonal matrix to realise
        
    Returns:
        QuantumCircuit which implements the unitary
    """
    #Check input
    nbits = np.log2(umat.shape[0])
    if umat.shape[0] != umat.shape[1] or not nbits.is_integer:
        raise Exception('Matrix has incorrect dimensions')
    elif not np.allclose(np.matmul(umat,np.conj(umat)), np.identity(umat.shape[0])):
        raise Exception('Matrix is not unitary')
    elif not np.allclose(umat, np.diag(np.diagonal(umat))):
        raise Exception('matrix is not diagonal')
    nbits = int(nbits) #n classical bits 1 < n < 8
    
    
    # Pauli Decompose 
    hmat = np.angle(umat) #Tells us the complex argument of the matrix, thereby parameterizing the circuit
    components = PauliDecompose(hmat) #Decomposing the circuit according to the decomposing method
    
    # order to implement Pauli component (reduces CNOTs used)
    # iteratively add each pauli component
    
    #Define the qubits in Cirq
    circuit = cirq.Circuit()
    
    
    circuit.append(cirq.I(q) for q in cirq.LineQubit.range(nbits))
    
    
    for operator, coeff in components.items():
        #find qubits CX-RZ-CX
        cxlist = []
        for i in range(len(operator)):
            cxlist.append(i) if operator[i] == 'Z' else None
        cxlist = [nbits - 1 - i for i in cxlist]
        if coeff == 0 or len(cxlist) == 0:
            continue
        elif len(cxlist) == 1:           
            circuit.append(cirq.ops.rz(-2*coeff).on(cirq.LineQubit(cxlist[0])))
        else: 
            # print(cxlist)
            # print(type(cxlist))
            for ctrl in cxlist[:-1]:
                target, control = cirq.LineQubit(cxlist[-1]), cirq.LineQubit(ctrl)
                # print(target, control)
                cnot = cirq.CNOT(control=control, target=target)
                circuit.append( cnot )
            circuit.append(cirq.ops.rz(-2*coeff).on(cirq.LineQubit(cxlist[-1])))
            for ctrl in reversed(cxlist[:-1]):
                circuit.append(cirq.ops.CNOT(control=cirq.LineQubit(ctrl), target=cirq.LineQubit(cxlist[-1])))
    return circuit  
                    
#Testing!
ckt = UGate(np.diag([-1,1,-1,1]))
print(ckt)

0: ───I───Rz(-π)───

1: ───I────────────


In [13]:
def Ugate_Sycamore(umat):
    """Realises the specified unitary digonal matrix in a Cirq quantum cricuit
    
    Parameters:
        umat (matrix): unitary diagonal matrix to realise
        
    Returns:
        QuantumCircuit which implements the unitary
    """
    #Check input
    nbits = np.log2(umat.shape[0])
    if umat.shape[0] != umat.shape[1] or not nbits.is_integer:
        raise Exception('Matrix has incorrect dimensions')
    elif not np.allclose(np.matmul(umat,np.conj(umat)), np.identity(umat.shape[0])):
        raise Exception('Matrix is not unitary')
    elif not np.allclose(umat, np.diag(np.diagonal(umat))):
        raise Exception('matrix is not diagonal')
    nbits = int(nbits) #n classical bits 1 < n < 8
    
    
    # Pauli Decompose 
    hmat = np.angle(umat) #Tells us the complex argument of the matrix, thereby parameterizing the circuit
    components = PauliDecompose(hmat) #Decomposing the circuit according to the decomposing method
    
    # order to implement Pauli component (reduces CNOTs used)
    # iteratively add each pauli component
    
    #Define the qubits in Cirq
    circuit = cirq.Circuit(device=cirq.google.Sycamore)
    circuit.append(cirq.I(q) for q in cirq.GridQubit.range(nbits))
    for operator, coeff in components.items():
        #find qubits CX-RZ-CX
        cxlist = []
        for i in range(len(operator)):
            cxlist.append(i) if operator[i] == 'Z' else None
        cxlist = [nbits - 1 - i for i in cxlist]
        if coeff == 0 or len(cxlist) == 0:
            continue
        elif len(cxlist) == 1:           
            circuit.append(cirq.ops.rz(-2*coeff).on(cirq.GridQubit(cxlist[0])))
        else: 
            # print(cxlist)
            # print(type(cxlist))
            for ctrl in cxlist[:-1]:
                target, control = cirq.GridQubit(cxlist[-1]), cirq.GridQubit(ctrl)
                # print(target, control)
                cnot = cirq.CNOT(control=control, target=target)
                circuit.append( cnot )
            circuit.append(cirq.ops.rz(-2*coeff).on(cirq.GridQubit(cxlist[-1])))
            for ctrl in reversed(cxlist[:-1]):
                circuit.append(cirq.ops.CNOT(control=cirq.GridQubit(ctrl), target=cirq.GridQubit(cxlist[-1])))
    return circuit  

    #Testing!
ckt = UGate(np.diag([-1,1,-1,1]))
print(ckt)

0: ───I───Rz(-π)───

1: ───I────────────


In [14]:
def UGate(umat):
    """Realises the specified unitary digonal matrix in a Cirq quantum cricuit
    
    Parameters:
        umat (matrix): unitary diagonal matrix to realise
        
    Returns:
        QuantumCircuit which implements the unitary
    """
    #Check input
    nbits = np.log2(umat.shape[0])
    if umat.shape[0] != umat.shape[1] or not nbits.is_integer:
        raise Exception('Matrix has incorrect dimensions')
    elif not np.allclose(np.matmul(umat,np.conj(umat)), np.identity(umat.shape[0])):
        raise Exception('Matrix is not unitary')
    elif not np.allclose(umat, np.diag(np.diagonal(umat))):
        raise Exception('matrix is not diagonal')
    nbits = int(nbits) #n classical bits 1 < n < 8
    
    
    # Pauli Decompose 
    hmat = np.angle(umat) #Tells us the complex argument of the matrix, thereby parameterizing the circuit
    components = PauliDecompose(hmat) #Decomposing the circuit according to the decomposing method
    
    # order to implement Pauli component (reduces CNOTs used)
    # iteratively add each pauli component
    
    #Define the qubits in Cirq
    circuit = cirq.Circuit()
    
    
    circuit.append(cirq.I(q) for q in cirq.LineQubit.range(nbits))
    
    
    for operator, coeff in components.items():
        #find qubits CX-RZ-CX
        cxlist = []
        for i in range(len(operator)):
            cxlist.append(i) if operator[i] == 'Z' else None
        cxlist = [nbits - 1 - i for i in cxlist]
        if coeff == 0 or len(cxlist) == 0:
            continue
        elif len(cxlist) == 1:           
            circuit.append(cirq.ops.rz(-2*coeff).on(cirq.LineQubit(cxlist[0])))
        else: 
            for ctrl in cxlist[:-1]:
                circuit.append(cirq.ops.CNOT(control=ctrl, target=cirq.LineQubit(cxlist[:-1])))
            circuit.append(cirq.ops.rz(-2*coeff).on(cirq.LineQubit(cxlist[-1])))
            for ctrl in reversed(cxlist[:-1]):
                circuit.append(cirq.ops.CNOT(control=ctrl, target=cirq.LineQubit(cxlist[:-1])))
    return circuit  
                    
#Testing!
ckt = UGate(np.diag([-1,1,-1,1]))
print(ckt)

0: ───I───Rz(-π)───

1: ───I────────────


In [15]:
qubits = cirq.GridQubit.square(4)
print(qubits)

[cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(0, 2), cirq.GridQubit(0, 3), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(1, 2), cirq.GridQubit(1, 3), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(2, 2), cirq.GridQubit(2, 3), cirq.GridQubit(3, 0), cirq.GridQubit(3, 1), cirq.GridQubit(3, 2), cirq.GridQubit(3, 3)]


In [16]:
print(cirq.google.Sycamore)

                                             (0, 5)───(0, 6)
                                             │        │
                                             │        │
                                    (1, 4)───(1, 5)───(1, 6)───(1, 7)
                                    │        │        │        │
                                    │        │        │        │
                           (2, 3)───(2, 4)───(2, 5)───(2, 6)───(2, 7)───(2, 8)
                           │        │        │        │        │        │
                           │        │        │        │        │        │
                  (3, 2)───(3, 3)───(3, 4)───(3, 5)───(3, 6)───(3, 7)───(3, 8)───(3, 9)
                  │        │        │        │        │        │        │        │
                  │        │        │        │        │        │        │        │
         (4, 1)───(4, 2)───(4, 3)───(4, 4)───(4, 5)───(4, 6)───(4, 7)───(4, 8)───(4, 9)
         │        │        │        │        │        │   

In [18]:
def UGate(umat):
    """Realises the specified unitary digonal matrix in a Cirq quantum cricuit
    
    Parameters:
        umat (matrix): unitary diagonal matrix to realise
        
    Returns:
        QuantumCircuit which implements the unitary
    """
    #Check input
    nbits = np.log2(umat.shape[0])
    if umat.shape[0] != umat.shape[1] or not nbits.is_integer:
        raise Exception('Matrix has incorrect dimensions')
    elif not np.allclose(np.matmul(umat,np.conj(umat)), np.identity(umat.shape[0])):
        raise Exception('Matrix is not unitary')
    elif not np.allclose(umat, np.diag(np.diagonal(umat))):
        raise Exception('matrix is not diagonal')
    nbits = int(nbits) #n classical bits 1 < n < 8
    
    
    # Pauli Decompose 
    hmat = np.angle(umat) #Tells us the complex argument of the matrix, thereby parameterizing the circuit
    components = PauliDecompose(hmat) #Decomposing the circuit according to the decomposing method
    
    # order to implement Pauli component (reduces CNOTs used)
    # iteratively add each pauli component
    
    #Define the qubits in Cirq
    circuit = cirq.Circuit()
    
    
    circuit.append(cirq.I(q) for q in cirq.LineQubit.range(nbits))
    
    
    for operator, coeff in components.items():
        #find qubits CX-RZ-CX
        cxlist = []
        for i in range(len(operator)):
            cxlist.append(i) if operator[i] == 'Z' else None
        cxlist = [nbits - 1 - i for i in cxlist]
        if coeff == 0 or len(cxlist) == 0:
            continue
        elif len(cxlist) == 1:           
            circuit.append(cirq.ops.rz(-2*coeff).on(cirq.LineQubit(cxlist[0])))
        else: 
            # print(cxlist)
            # print(type(cxlist))
            for ctrl in cxlist[:-1]:
                target, control = cirq.LineQubit(cxlist[-1]), cirq.LineQubit(ctrl)
                # print(target, control)
                cnot = cirq.CNOT(control=control, target=target)
                circuit.append( cnot )
            circuit.append(cirq.ops.rz(-2*coeff).on(cirq.LineQubit(cxlist[-1])))
            for ctrl in reversed(cxlist[:-1]):
                circuit.append(cirq.ops.CNOT(control=cirq.LineQubit(ctrl), target=cirq.LineQubit(cxlist[-1])))
    return circuit  
                    
#Testing!
ckt = UGate(np.diag([-1,1,-1,1]))
print(ckt)

0: ───I───Rz(-π)───

1: ───I────────────


In [19]:
nbits = 5
# generate random unitary diagonal matrix
umat = np.random.rand(2**nbits)
umat = np.exp(2j * np.pi * umat)
umat = np.diag(umat)

print(np.shape(umat))
# simulate generated circuit

qc = UGate(umat)

#backend = Aer.get_backend('unitary_simulator')
#job = execute(qc, backend)
#out = job.result().get_unitary()

# compare input & output
print("input U: ", np.diagonal(umat))
#print("output U: ", np.diagonal(out))
#print("identical: ", np.allclose(umat, out))
print(qc)

(32, 32)
input U:  [ 0.04284379-0.99908178j  0.75335844-0.65761011j -0.86759207-0.49727658j
 -0.20711737+0.9783161j   0.64719147+0.76232749j  0.63242665-0.77462025j
  0.98761974+0.15686697j -0.60099882+0.79924991j  0.03691372+0.99931846j
 -0.6529381 -0.75741127j  0.96311242-0.26909936j -0.99122115+0.1322143j
  0.57446068+0.81853218j  0.89255988-0.45092888j  0.99762227-0.06891886j
 -0.71185811-0.70232331j -0.21086154-0.97751594j -0.69076292-0.72308132j
 -0.32363552-0.94618183j  0.3651008 +0.93096799j -0.83428571+0.55133234j
 -0.87577029-0.48272808j  0.99303144-0.11784975j -0.39405052-0.91908878j
 -0.55151553-0.83416462j -0.9858429 -0.16767164j -0.98454791+0.17511541j
  0.95110266-0.30887493j  0.91598297+0.40121715j -0.49589399-0.86838306j
  0.69076302+0.72308122j -0.66041888-0.7508974j ]
0: ───I───Rz(-0.258π)───X───Rz(-0.349π)───X───X───Rz(0.382π)───X────────────────────────X───X───Rz(-0.123π)───X───X───X───Rz(0.298π)───X─────────────────────────X───X───Rz(-0.223π)───X───X──────────────